In [2]:
import numpy as np

In [146]:
class PSO:
    def __init__(self, max_evals=10*5):
        self.posicion_de_cada_elemento = np.random.uniform(-5.12, 5.12, size=(50,10)) #Pos
        self.velocidad_de_cada_elemento = np.random.uniform(0, 10, size=(50,10)) #vel
        self.mejor_fitness_de_cada_elemento = np.zeros((50, 10))
        
        self.gBest = -1
        self.bestFitness = [np.inf] * 10

        self.max_evals=max_evals
        
    def actualizar_posiciones(self):
        formula = 0.7 * self.velocidad_de_cada_elemento[:] + 1.5 * np.random.random() * (self.mejor_fitness_de_cada_elemento[:] 
                    - self.posicion_de_cada_elemento[:]) + 1.5 * np.random.random() * (self.posicion_de_cada_elemento[self.gBest] - self.posicion_de_cada_elemento[:])         
        self.posicion_de_cada_elemento[:] += formula
        self.velocidad_de_cada_elemento = formula

    def get_modularity(self, solution):
        return np.sum(self.posicion_de_cada_elemento[solution]**2)

    def run(self):
        evals = 0
        while evals < self.max_evals:
            evals += 1
            #Actualizamos mejores modularidades y nos quedamos con el mejor
            for i in range(50):
                lag = [self.get_modularity(i)] * 10
                if lag[0] < self.mejor_fitness_de_cada_elemento[i][0]:
                    self.mejor_fitness_de_cada_elemento[i] = lag
                if lag < self.bestFitness:
                    self.gBest = i
                    self.bestFitness = lag
                    print(f"Mejor fitnes actualizado: {self.bestFitness[0]} conseguido por la particula {self.gBest}")

            self.actualizar_posiciones()

In [147]:
'''
1. Poblacion 50 arrays de 10 elementos
2. For
3. Calculamos modularidad de cada elemento
4. Actualizamos al lider y mejor recuerdo de cada uno
5. Movemos los puntos
6. Repetimos hasta ...
'''

'\n1. Poblacion 50 arrays de 10 elementos\n2. For\n3. Calculamos modularidad de cada elemento\n4. Actualizamos al lider y mejor recuerdo de cada uno\n5. Movemos los puntos\n6. Repetimos hasta ...\n'

In [148]:
pso = PSO()

In [149]:
pso.run()

Mejor fitnes actualizado: 76.84392997245669 conseguido por la particula 0
Mejor fitnes actualizado: 75.79257176120618 conseguido por la particula 1
Mejor fitnes actualizado: 60.37245650839118 conseguido por la particula 2
Mejor fitnes actualizado: 36.8511620340254 conseguido por la particula 9
Mejor fitnes actualizado: 8.909774615409356 conseguido por la particula 9
Mejor fitnes actualizado: 4.418742815049756 conseguido por la particula 0
Mejor fitnes actualizado: 3.793958181664269 conseguido por la particula 1
Mejor fitnes actualizado: 3.0121511647284684 conseguido por la particula 5
Mejor fitnes actualizado: 2.932826453808683 conseguido por la particula 9
Mejor fitnes actualizado: 1.1290286694965037 conseguido por la particula 11
Mejor fitnes actualizado: 0.16373147975186575 conseguido por la particula 11
Mejor fitnes actualizado: 0.002364505325406198 conseguido por la particula 11
Mejor fitnes actualizado: 0.001275703472179495 conseguido por la particula 11


In [31]:
np.random.random()

0.6810113668643034

In [150]:
import numpy as np

class PSO_Exam:
    def __init__(self, n_particles=50, n_dim=10, max_evals=100000):
        self.n_particles = n_particles
        self.n_dim = n_dim
        self.max_evals = max_evals
        
        # --- 1. INICIALIZACIÓN ---
        # Rango [-5.12, 5.12]
        self.X = np.random.uniform(-5.12, 5.12, (n_particles, n_dim))
        # Velocidad inicial pequeña (ceros o random pequeño)
        self.V = np.random.randn(n_particles, n_dim) * 0.1
        
        # --- 2. MEMORIA (pBest y gBest) ---
        # Mejor posición personal (pBest): Al principio es la actual
        self.pBest_pos = self.X.copy()
        # Mejor nota personal: Al principio es la actual
        self.pBest_val = np.array([self.get_fitness(ind) for ind in self.X])
        
        # Mejor global (gBest): El mejor de los personales
        best_idx = np.argmin(self.pBest_val)
        self.gBest_pos = self.pBest_pos[best_idx].copy()
        self.gBest_val = self.pBest_val[best_idx]
        
        # Hiperparámetros del enunciado
        self.w = 0.7  # Inercia
        self.c1 = 1.5 # Cognitivo
        self.c2 = 1.5 # Social

    def get_fitness(self, particle):
        """
        Función Sphere: Suma de cuadrados.
        Recibe un array (10,) y devuelve un float.
        """
        return np.sum(particle**2)

    def update_particles(self):
        """
        El Motor Físico: Mueve todo el enjambre a la vez usando matrices.
        """
        # Generamos aleatorios (r1, r2) para TODAS las coordenadas de golpe
        r1 = np.random.rand(self.n_particles, self.n_dim)
        r2 = np.random.rand(self.n_particles, self.n_dim)
        
        # --- FÓRMULA DE VELOCIDAD (Vectorizada) ---
        # v_new = w*v + c1*r1*(pBest - X) + c2*r2*(gBest - X)
        inertia = self.w * self.V
        cognitive = self.c1 * r1 * (self.pBest_pos - self.X)
        social = self.c2 * r2 * (self.gBest_pos - self.X) # Numpy resta vector gBest a cada fila de X
        
        self.V = inertia + cognitive + social
        
        # --- FÓRMULA DE POSICIÓN ---
        self.X = self.X + self.V
        
        # Opcional: Limitar al rango (rebotar o clippear) para no salir del mapa
        self.X = np.clip(self.X, -5.12, 5.12)

    def run(self):
        evals = 0
        iteration = 0
        
        # Como evaluamos 50 partículas por ciclo, avanzamos de 50 en 50
        while evals < self.max_evals:
            iteration += 1
            
            # 1. MOVER ENJAMBRE
            self.update_particles()
            
            # 2. EVALUAR Y ACTUALIZAR MEMORIA
            # Calculamos fitness de las nuevas posiciones
            # (Vectorizado: aplicamos la función a cada fila)
            current_fitness_values = np.array([self.get_fitness(p) for p in self.X])
            evals += self.n_particles
            
            # 3. ACTUALIZAR pBest (Personal)
            # Buscamos dónde hemos mejorado (máscara booleana)
            improved_indices = current_fitness_values < self.pBest_val
            
            # Actualizamos solo donde hubo mejora
            self.pBest_pos[improved_indices] = self.X[improved_indices]
            self.pBest_val[improved_indices] = current_fitness_values[improved_indices]
            
            # 4. ACTUALIZAR gBest (Global)
            # Miramos si alguien de la nueva generación ha superado el récord mundial
            min_val_iter = np.min(self.pBest_val)
            min_idx_iter = np.argmin(self.pBest_val)
            
            if min_val_iter < self.gBest_val:
                self.gBest_val = min_val_iter
                self.gBest_pos = self.pBest_pos[min_idx_iter].copy()
                print(f"Iter {iteration} (Evals {evals}): Nuevo Récord Global -> {self.gBest_val:.10f}")
                
        return self.gBest_val

# --- TEST DEL EXAMEN ---
pso_solver = PSO_Exam()
resultado = pso_solver.run()
print(f"\nResultado final (debe ser cercano a 0): {resultado}")

Iter 1 (Evals 50): Nuevo Récord Global -> 23.0579934781
Iter 3 (Evals 150): Nuevo Récord Global -> 21.6215230284
Iter 4 (Evals 200): Nuevo Récord Global -> 20.6035645727
Iter 6 (Evals 300): Nuevo Récord Global -> 20.3839647267
Iter 7 (Evals 350): Nuevo Récord Global -> 13.9110888959
Iter 8 (Evals 400): Nuevo Récord Global -> 6.6695877660
Iter 9 (Evals 450): Nuevo Récord Global -> 4.6111033805
Iter 11 (Evals 550): Nuevo Récord Global -> 4.3618832535
Iter 12 (Evals 600): Nuevo Récord Global -> 3.2414913277
Iter 14 (Evals 700): Nuevo Récord Global -> 2.5134658738
Iter 15 (Evals 750): Nuevo Récord Global -> 2.3932447714
Iter 16 (Evals 800): Nuevo Récord Global -> 2.0838360392
Iter 17 (Evals 850): Nuevo Récord Global -> 1.6220844370
Iter 19 (Evals 950): Nuevo Récord Global -> 0.8900618853
Iter 21 (Evals 1050): Nuevo Récord Global -> 0.8353620200
Iter 22 (Evals 1100): Nuevo Récord Global -> 0.7865472602
Iter 25 (Evals 1250): Nuevo Récord Global -> 0.6788046213
Iter 26 (Evals 1300): Nuevo Réc